In [1]:
import pandas as pd
import numpy as np
import json
import urllib.request
import urllib.parse
import time
from datetime import datetime
import pickle
import sys, os
from io import StringIO
import re

## Figi API

In [2]:
#PUT YOUR OPENFIGI KEY HERER
openfigi_apikey = "dc3a3d2d-9e29-465c-b062-49b60c434eba"

In [3]:
jobs = [
    {'idType': 'ID_CUSIP', 'idValue': '00206R102'}
]

In [4]:
def map_jobs(jobs):
    '''
    Send an collection of mapping jobs to the API in order to obtain the
    associated FIGI(s).
    Parameters
    ----------
    jobs : list(dict)
        A list of dicts that conform to the OpenFIGI API request structure. See
        https://www.openfigi.com/api#request-format for more information. Note
        rate-limiting requirements when considering length of `jobs`.
    Returns
    -------
    list(dict)
        One dict per item in `jobs` list that conform to the OpenFIGI API
        response structure.  See https://www.openfigi.com/api#response-fomats
        for more information.
    '''
    handler = urllib.request.HTTPHandler()
    opener = urllib.request.build_opener(handler)
    openfigi_url = 'https://api.openfigi.com/v2/mapping'
    request = urllib.request.Request(openfigi_url, data=bytes(json.dumps(jobs), encoding='utf-8'))
    request.add_header('Content-Type','application/json')
    if openfigi_apikey:
        request.add_header('X-OPENFIGI-APIKEY', openfigi_apikey)
    request.get_method = lambda: 'POST'
    connection = opener.open(request)
    if connection.code != 200:
        raise Exception('Bad response code {}'.format(str(response.status_code)))
    return json.loads(connection.read().decode('utf-8'))


def search_jobs(jobs):
    '''
    Send an collection of searching jobs to the API in order to obtain the
    associated FIGI(s).
    Parameters
    ----------
    jobs : list(dict)
        A list of dicts that conform to the OpenFIGI API request structure. See
        https://www.openfigi.com/api#request-format for more information. Note
        rate-limiting requirements when considering length of `jobs`.
    Returns
    -------
    list(dict)
        One dict per item in `jobs` list that conform to the OpenFIGI API
        response structure.  See https://www.openfigi.com/api#response-fomats
        for more information.
    '''
    handler = urllib.request.HTTPHandler()
    opener = urllib.request.build_opener(handler)
    openfigi_url = 'https://api.openfigi.com/v2/search'
    request = urllib.request.Request(openfigi_url, data=bytes(json.dumps(jobs), encoding='utf-8'))
    request.add_header('Content-Type','application/json')
    if openfigi_apikey:
        request.add_header('X-OPENFIGI-APIKEY', openfigi_apikey)
    request.get_method = lambda: 'POST'
    connection = opener.open(request)
    if connection.code != 200:
        raise Exception('Bad response code {}'.format(str(response.status_code)))
    return json.loads(connection.read().decode('utf-8'))

def job_results_handler(jobs, job_results):
    '''
    Handle the `map_jobs` results.  See `map_jobs` definition for more info.
    Parameters
    ----------
    jobs : list(dict)
        The original list of mapping jobs to perform.
    job_results : list(dict)
        The results of the mapping job.
    Returns
    -------
        None
    '''
    for job, result in zip(jobs, job_results):
        job_str = '|'.join(job.values())
        figis_str = ','.join([d['figi'] for d in result.get('data', [])])
        result_str = figis_str or result.get('error')
        output = '%s maps to FIGI(s) ->\n%s\n---' % (job_str, result_str)
        print(output)
    return jobs


def main():
    '''
    Map the defined `jobs` and handle the results.
    Returns
    -------
        None
    '''
    job_results = map_jobs(jobs)
    job_results_handler(jobs, job_results)

In [5]:
main()

ID_CUSIP|00206R102 maps to FIGI(s) ->
BBG000BSJK37,BBG000BSJKF4,BBG000BSJL53,BBG000BSJLV4,BBG000BSJMD2,BBG000BSJND0,BBG000BSJNL1,BBG000BSJNX8,BBG000BSJP63,BBG000BSJSM9,BBG000BSJTX5,BBG000BSJV97,BBG000BSJVS6,BBG000BSJW86,BBG000C1YTY5,BBG000FWN729,BBG000FWN7K9,BBG000FWN7Y4,BBG000FWN8K7,BBG000FWN9H9,BBG000FWNB66,BBG000FWNCR1,BBG000FWNDT7,BBG000FWNFG6,BBG000HW5DB4,BBG000HW5DZ8,BBG000QFT5K2,BBG000RFRWN1,BBG000RYPVH1,BBG001HRHZQ5,BBG001HRHZR4,BBG002H8MWV0,BBG0032FL995,BBG0032FL9B2,BBG0032FL9C1,BBG006M6YBB4,BBG0077GTLW8,BBG007F5VFJ8,BBG007F5VFM4,BBG007F5VFP1,BBG007F5VFQ0,BBG007F5VFR9,BBG007F5VFS8,BBG007F5VFT7,BBG007F5VFV4,BBG007F5VFW3,BBG007F5VFY1,BBG007F5VG06,BBG007F5VG15,BBG007F5VG24,BBG007F5VG33,BBG007F5VG42,BBG007F5VG51,BBG007F5VG79,BBG007F5VG97,BBG0089KKK01,BBG0089KKK38,BBG0089KKK56,BBG0089KKK65,BBG0089KKK74,BBG0089KKK83,BBG0089KKK92,BBG0089KKKB9,BBG0089KKKC8,BBG0089KKKF5,BBG0089KKKH3,BBG0089KKKJ1,BBG0089KKKK9,BBG0089KKKL8,BBG0089KKKM7,BBG0089KKKN6,BBG0089KKKQ3,BBG0089KKKS1,BBG009STMZY7,

# Figi Collect

In [6]:
sys.path.insert(0, '/content/drive/MyDrive/IEORIndustrialProject/figi_yahoo_model')

DIR = os.path.abspath(os.getcwd())

"""
Security Id type used
"""
idTypes = {
    'Cusip': 'ID_CUSIP',
    'Isin': 'ID_ISIN',
    'TickerSymbol': 'TICKER'
}

"""
A helper function to construct a request to query data based on Ids (CUSIP/ISIN/SEDOL)
"""
def construct_jobs_mapping(Ids):
    """
    Ids: dict
        dictionary of list of ids. Key is the id type, and value is list of ids
    Return
        dict: job request used to query data from openfigi
    """
    jobs = []
    for IdType in Ids:
        for id in Ids[IdType]:
            job = dict()
            job['idType'] = IdType
            job['idValue'] = id
            jobs.append(job)
    return jobs

"""
read output from openfigi data
"""
def parse_output_extract_first(output):
    """
    output: dict
        Output dict from reading openfigi json file
    Return 
        list: the first chunk of info as a list
    """
    ret = []
    if output:
        info = output[0]['data'][0]
        return [info['name'], info['ticker'], info['securityType'], info['securityType2'], info['marketSector'], info['securityDescription']]

"""
A helper function to get the type based on Ids
"""
def getOutput(IdType, idList):
    """
    IdType: str
        String that indicates the id type. Use defined idTypes
    idList: list
        list that contains ids of a specific type
    Return
        dict: key=>id , value=> list of two types from openfigi
    """
    idInfo = {}
    jobsList = []
    counter = 0
    notwork = []
    for id in idList:
        counter += 1
        jobsList.append(id)
        if counter % 10 == 0:
            print(counter)
            time.sleep(2)
        try:
            requestJobs = construct_jobs_mapping({IdType:jobsList})
            output = map_jobs(requestJobs)
            idInfo[jobsList[0]] = parse_output_extract_first(output)
        except:
            idInfo[jobsList[0]] = ["NotFound", "NotFound", "NotFound", "NotFound", "NotFound", "NotFound"]
            # notwork.append(id)
        # for i in len(output):
        # cusipinfo[jobsList[i]] = parse_output_extract_first(output)
        jobsList = []
    return idInfo

"""
Get figi type from pandas dataframe
"""
def figi_from_pd(df, IdType):
    """
    df: pandas dataframe
        A pandas dataframe with three columns of different ids
    IdType: list
        list of IdType. Prefer 'Cusip', 'Isin', 'TickerSymbol'.
    Return
        
    """
    d = df[['Cusip', 'Isin', 'TickerSymbol']].drop_duplicates()

    cusip_list = df['Cusip'].unique()
    isin_list = df['Isin'].unique()
    ticker_symbols = df['TickerSymbol'].unique()

    cusipinfo = getOutput('ID_CUSIP', cusip_list.tolist())
    isinInfo = getOutput('ID_ISIN', isin_list.tolist())
    tickerSymbolInfo = getOutput('TICKER', ticker_symbols.tolist())

    cusip_df = construct_df('cusip', cusipinfo)
    isin_df = construct_df('isin', isinInfo)
    ticker_df = construct_df('tickerSymbol', tickerSymbolInfo)

    current_time = datetime.now().strftime("%H%M%S")
    cusip_df.to_csv(DIR + "/cusip_type_figi" + current_time + ".csv")
    isin_df.to_csv(DIR + "/isin_type_figi"+current_time+".csv")
    ticker_df.to_csv(DIR + "/ticker_type_figi"+current_time+".csv")

    return [cusip_df, isin_df, ticker_df]


def construct_df(idType, InfoDict):
    df_init = {idType:[], 
               'name':[],
               'ticker':[],
               'type1':[], 
               'type2':[],
              'sector':[],
              'description':[]}
    for id in InfoDict:
        df_init[idType].append(id)
        temp = InfoDict[id]
        df_init['name'].append(temp[0])
        df_init['ticker'].append(temp[1])
        df_init['type1'].append(temp[2])
        df_init['type2'].append(temp[3])
        df_init['sector'].append(temp[4])
        df_init['description'].append(temp[5])
    return pd.DataFrame(data=df_init)

# Data clearing

In [7]:
import csv

with open('/Users/aeriliu/Documents/Columbia/Spring2021/IEOR4511/New Vehicle Data/withclosingprice/0023.txt', 'r') as in_file:
    stripped = (line.strip() for line in in_file)
    lines = (line.split(";") for line in stripped if line)
    with open('/Users/aeriliu/Documents/Columbia/Spring2021/IEOR4511/New Vehicle Data/withclosingprice/0023.csv', 'w') as out_file:
        writer = csv.writer(out_file)
        writer.writerows(lines)

In [8]:
with open('/Users/aeriliu/Documents/Columbia/Spring2021/IEOR4511/New Vehicle Data/withclosingprice/0018.txt', 'r') as in_file:
    stripped = (line.strip() for line in in_file)
    lines = (line.split(";") for line in stripped if line)
    with open('/Users/aeriliu/Documents/Columbia/Spring2021/IEOR4511/New Vehicle Data/withclosingprice/0018.csv', 'w') as out_file:
        writer = csv.writer(out_file)
        writer.writerows(lines)

In [9]:
schwab = pd.read_csv('/Users/aeriliu/Documents/Columbia/Spring2021/IEOR4511/New Vehicle Data/withclosingprice/0023.csv')
mssb = pd.read_csv('/Users/aeriliu/Documents/Columbia/Spring2021/IEOR4511/New Vehicle Data/withclosingprice/0018.csv')

In [10]:
schwab.head()

,Recordtype,Custdianid,Mstracctnumber,Masteraccountname,Prodcode,Prodcatgcode,Taxcode,Lyst,Tickersymbol,Industrytickersymbol,...,Versmrkr1,PI,OI,Versmrkr2,Factor,Factordate,F1,IDArchiveMaster,ClosingPrice,WTType
0,D1,SCHWAB,8275759,SEVENONESEVEN CAPITAL MANAGEME,COM,EQTY,A,cs,ALRM,ALRM,...,VER01.01,N,N,VER01.02,0.0,NaN,NaN,10778,92.04,CSUS
1,D1,SCHWAB,8275759,SEVENONESEVEN CAPITAL MANAGEME,COM,EQTY,A,cs,BIG,BIG,...,VER01.01,N,N,VER01.02,0.0,NaN,NaN,10778,66.54,CSUS
2,D1,SCHWAB,8275759,SEVENONESEVEN CAPITAL MANAGEME,COM,EQTY,A,cs,ENPH,ENPH,...,VER01.01,N,N,VER01.02,0.0,NaN,NaN,10778,165.34,CSUS
3,D1,SCHWAB,8275759,SEVENONESEVEN CAPITAL MANAGEME,COM,EQTY,A,cs,FCX,FCX,...,VER01.01,N,N,VER01.02,0.0,NaN,NaN,10778,37.47,CSUS
4,D1,SCHWAB,8275759,SEVENONESEVEN CAPITAL MANAGEME,COM,EQTY,A,cs,HIBB,HIBB,...,VER01.01,N,N,VER01.02,0.0,NaN,NaN,10778,64.99,CSUS


In [11]:
schwab.columns

Index(['Recordtype', 'Custdianid', 'Mstracctnumber', 'Masteraccountname',
       'Prodcode', 'Prodcatgcode', 'Taxcode', 'Lyst', 'Tickersymbol',
       'Industrytickersymbol', 'CUSIP', 'Schwabsecnbr', 'Reorgsecnbr',
       'Rulstsufid', 'ISIN', 'SEDOL', 'Optionsdisplaysymbol',
       'Securitydescriptionline1', 'Securitydescriptionline2',
       'Securitydescriptionline3', 'Scrtydesline4', 'Underlyingtickersymbol',
       'Underlyingindustrytickersymbol', 'Underlyngcusip', 'Underlyschwab',
       'Underlyingitmissid', 'Unrulsufid', 'Underlyingisin', 'Underlysedol',
       'Mnymkcode', 'SF', 'Securityvaluationunit', 'Optnrtsymbol',
       'Optexprdate', 'CP', 'Interestrate', 'Maturitydate', 'Tipsfactor',
       'Assetbackedfactor', 'Facevalueamt', 'Stcd', 'Versmrkr1', 'PI', 'OI',
       'Versmrkr2', 'Factor', 'Factordate', 'F1', 'IDArchiveMaster',
       'ClosingPrice', 'WTType'],
      dtype='object')

In [12]:
mssb.columns

Index(['Cusip', 'SymbolOsiSymbol', 'SecurityDescription', 'UnderlyingCusip',
       'Sedol', 'Isin', 'SPRating', 'MoodyRating', 'CurrentFace',
       'OriginalFace', 'CurrentMbsFactor', 'CurrentMbsFactorDate', 'Currency',
       'IssueDate', 'DatedDate', 'PayDate', 'MaturityDate', 'DayCount',
       'PayFrequency', 'StateCode', 'CountryCode', 'ExchangeCode', 'TaxCode',
       'MaturityType', 'ExDate', 'RecordDate', 'SecurityCode', 'CouponRate',
       'ShareMultiplier', 'ProductId', 'RestrictFlag', 'IntrumentType',
       'BondType', 'BondStatusInd', 'AlternateSecurityIndicator', 'Filler',
       'AdvSecurityType', 'MaturityDate2', 'Filler2', 'idarchivemaster',
       'ClosePrice', 'WTType'],
      dtype='object')

# Figi Data Collect

In [24]:
mssb_df = mssb[['Cusip', 'Isin', 'SymbolOsiSymbol']]
mssb_df.columns = ['Cusip', 'Isin', 'TickerSymbol']

In [25]:
schwab_df = schwab[['CUSIP', 'ISIN', 'Tickersymbol']]
schwab_df.columns = ['Cusip', 'Isin', 'TickerSymbol']

In [26]:
mssb_df.head()

,Cusip,Isin,TickerSymbol
0,001084102,US0010841023,AGCO
1,00751Y106,US00751Y1064,AAP
2,00912XAT1,US00912XAT19,NaN
3,026874156,US0268741560,NaN
4,052476F52,NaN,NaN


In [27]:
schwab_df.head()

,Cusip,Isin,TickerSymbol
0,011642105,US0116421050,ALRM
1,089302103,US0893021032,BIG
2,29355A107,US29355A1079,ENPH
3,35671D857,US35671D8570,FCX
4,428567101,US4285671016,HIBB


In [28]:
mssb_output = figi_from_pd(mssb_df, ['Cusip', 'Isin', 'TickerSymbol'])

10
20
30
40
50
60
70
80
90
100
110
120
130
140
150
160
170
180
190
200
210
220
230
240
250
260
270
280
290
300
310
320
330
340
350
360
370
380
390
400
410
420
430
440
450
460
470
480
490
500
510
520
530
540
550
560
570
580
590
600
610
620
630
640
650
660
670
680
690
700
710
720
730
740
750
760
770
780
790
800
810
820
830
840
850
860
870
880
890
900
910
920
930
940
950
960
970
980
990
1000
1010
1020
1030
1040
1050
1060
1070
1080
1090
1100
1110
1120
1130
1140
1150
1160
1170
1180
1190
1200
1210
1220
1230
1240
1250
1260
1270
1280
1290
1300
1310
1320
1330
1340
1350
1360
1370
1380
1390
1400
1410
1420
1430
1440
1450
1460
1470
1480
1490
1500
1510
1520
1530
1540
1550
1560
1570
1580
1590
1600
1610
1620
1630
1640
1650
1660
1670
1680
1690
1700
1710
1720
1730
1740
1750
1760
1770
1780
1790
1800
1810
1820
1830
1840
1850
1860
1870
1880
1890
1900
1910
1920
1930
1940
1950
1960
1970
1980
1990
2000
2010
2020
2030
2040
2050
2060
2070
2080
2090
2100
2110
2120
2130
10
20
30
40
50
60
70
80
90
100
110
120
130


In [29]:
schwab_output = figi_from_pd(schwab_df, ['Cusip', 'Isin', 'TickerSymbol'])

10
20
30
40
50
60
70
80
90
100
110
120
130
140
150
160
170
180
190
200
210
220
230
240
250
260
270
280
290
300
310
320
330
340
350
360
370
380
390
400
410
420
430
440
450
460
470
480
490
500
510
520
530
540
550
560
570
580
590
600
610
620
630
640
650
660
670
680
690
700
710
720
730
740
750
760
770
780
790
800
810
820
830
840
850
860
870
880
890
900
910
920
930
940
950
960
970
980
990
1000
1010
1020
1030
1040
1050
1060
1070
1080
1090
1100
1110
1120
1130
1140
1150
1160
1170
1180
1190
1200
1210
1220
1230
1240
1250
1260
1270
1280
1290
1300
1310
1320
1330
1340
1350
1360
1370
1380
1390
1400
1410
1420
1430
1440
1450
1460
1470
1480
1490
1500
1510
1520
1530
1540
1550
1560
1570
1580
1590
1600
1610
1620
1630
1640
1650
1660
1670
1680
1690
1700
1710
1720
1730
1740
1750
1760
1770
1780
1790
1800
1810
1820
1830
1840
1850
1860
1870
1880
1890
1900
1910
1920
1930
1940
1950
1960
1970
1980
1990
2000
2010
2020
2030
2040
2050
2060
2070
2080
2090
2100
2110
2120
2130
2140
2150
2160
2170
2180
2190
2200
2210
222

1590
1600
1610
1620
1630
1640
1650
1660
1670
1680
1690
1700
1710
1720
1730
1740
1750
1760
1770
1780
1790
1800
1810
1820
1830
1840
1850
1860
1870
1880
1890
1900
1910
1920
1930
1940
1950
1960
1970
1980
1990
2000
2010
2020
2030
2040
2050
2060
2070
2080
2090
2100
2110
2120
2130
2140
2150
2160
2170
2180
2190
2200
2210
2220
2230
2240
2250
2260
2270
2280
2290
2300
2310
2320
2330
2340
2350
2360
2370
2380
2390
2400
2410
2420
2430
2440
2450
2460
2470
2480
2490
2500
2510
2520
2530
2540
2550
2560
2570
2580
2590
2600
2610
2620
2630
2640
2650
2660
2670
2680
2690
2700
2710
2720
2730
2740
2750
2760
2770
2780
2790
2800
2810
2820
2830
2840
2850
2860
2870
2880
2890
2900
2910
2920
2930
2940
2950
2960
2970
2980
2990
3000
3010
3020
3030
3040
3050
3060
3070
3080
3090
3100
3110
3120
3130
3140
3150
3160
3170
3180
3190
3200
3210
3220
3230
3240
3250


In [30]:
mssb_cusip = pd.read_csv('/Users/aeriliu/Documents/Columbia/Spring2021/IEOR4511/cusip_type_figi_0018.csv')
schwab_cusip = pd.read_csv('/Users/aeriliu/Documents/Columbia/Spring2021/IEOR4511/cusip_type_figi_0023.csv')
mssb_isin = pd.read_csv('/Users/aeriliu/Documents/Columbia/Spring2021/IEOR4511/isin_type_figi_0018.csv')
schwab_isin = pd.read_csv('/Users/aeriliu/Documents/Columbia/Spring2021/IEOR4511/isin_type_figi_0023.csv')
mssb_ticker = pd.read_csv('/Users/aeriliu/Documents/Columbia/Spring2021/IEOR4511/ticker_type_figi_0018.csv')
schwab_ticker = pd.read_csv('/Users/aeriliu/Documents/Columbia/Spring2021/IEOR4511/ticker_type_figi_0023.csv')

In [31]:
mssb_cusip = mssb_cusip.drop(columns=['Unnamed: 0'])
schwab_cusip = schwab_cusip.drop(columns=['Unnamed: 0'])
mssb_isin = mssb_isin.drop(columns=['Unnamed: 0'])
mssb_ticker = mssb_ticker.drop(columns=['Unnamed: 0'])
schwab_isin = schwab_isin.drop(columns=['Unnamed: 0'])
schwab_ticker = schwab_ticker.drop(columns=['Unnamed: 0'])

In [34]:
mssb_ticker

,tickerSymbol,name,ticker,type1,type2,sector,description
0,AGCO,AGCO CORP,AGCO,Common Stock,Common Stock,Equity,AGCO
1,AAP,ARIHANT AGRO PRODUCTS LTD,AAP,Common Stock,Common Stock,Equity,AAP
2,NaN,NotFound,NotFound,NotFound,NotFound,NotFound,NotFound
3,ADSK,AUTODESK INC,ADSK,Common Stock,Common Stock,Equity,ADSK
4,AVB,NotFound,NotFound,NotFound,NotFound,NotFound,NotFound
...,...,...,...,...,...,...,...
934,LSTR,LANDSTAR SYSTEM INC,LSTR,Common Stock,Common Stock,Equity,LSTR
935,APAM,APERAM,APAM,Common Stock,Common Stock,Equity,APAM
936,EPAC,ENERPAC TOOL GROUP CORP,EPAC,Common Stock,Common Stock,Equity,EPAC
937,NEAR,BLACKROCK SHORT MATURITY BON,NEAR,ETP,Mutual Fund,Equity,NEAR


In [35]:
mssb_df.columns = ['cusip', 'isin', 'tickerSymbol']
schwab_df.columns = ['cusip', 'isin', 'tickerSymbol']

In [36]:
merged_mssb = pd.merge(mssb_df, mssb_cusip, how='left')
merged_schwab = pd.merge(schwab_df, schwab_cusip, how='left')

In [37]:
merged_mssb.to_csv('/Users/aeriliu/Documents/Columbia/Spring2021/IEOR4511/New Vehicle Data/merged_mssb_cusip.csv')
merged_schwab.to_csv('/Users/aeriliu/Documents/Columbia/Spring2021/IEOR4511/New Vehicle Data/merged_schwab_cusip.csv')

In [38]:
mssb_isin.set_index('isin', inplace = True)
schwab_isin.set_index('isin', inplace = True)

In [39]:
mssb_ticker.set_index('tickerSymbol', inplace = True)
schwab_ticker.set_index('tickerSymbol', inplace = True)

In [40]:
def isinmerge(ogdf, isindf):
    for i in range(len(ogdf)):
        if ogdf['name'][i]=='NotFound':
            if ogdf['isin'][i]:
                ogdf['name'][i]=isindf.loc[ogdf['isin'][i], 'name']
                ogdf['ticker'][i]=isindf.loc[ogdf['isin'][i], 'ticker']
                ogdf['type1'][i]=isindf.loc[ogdf['isin'][i], 'type1']
                ogdf['type2'][i]=isindf.loc[ogdf['isin'][i], 'type2']
                ogdf['sector'][i]=isindf.loc[ogdf['isin'][i], 'sector']
                ogdf['description'][i]=isindf.loc[ogdf['isin'][i], 'description']
    return ogdf

def tickermerge(ogdf, tickerdf):
    for i in range(len(ogdf)):
        if ogdf['name'][i]=='NotFound':
            if ogdf['tickerSymbol'][i]:
                ogdf['name'][i]=tickerdf.loc[ogdf['tickerSymbol'][i], 'name']
                ogdf['ticker'][i]=tickerdf.loc[ogdf['tickerSymbol'][i], 'ticker']
                ogdf['type1'][i]=tickerdf.loc[ogdf['tickerSymbol'][i], 'type1']
                ogdf['type2'][i]=tickerdf.loc[ogdf['tickerSymbol'][i], 'type2']
                ogdf['sector'][i]=tickerdf.loc[ogdf['tickerSymbol'][i], 'sector']
                ogdf['description'][i]=tickerdf.loc[ogdf['tickerSymbol'][i], 'description']
    return ogdf

In [42]:
isinmerge(merged_mssb, mssb_isin)

,cusip,isin,tickerSymbol,name,ticker,type1,type2,sector,description
0,001084102,US0010841023,AGCO,AGCO CORP,AGCO,Common Stock,Common Stock,Equity,AGCO
1,00751Y106,US00751Y1064,AAP,ADVANCE AUTO PARTS INC,AWN,Common Stock,Common Stock,Equity,AWN
2,00912XAT1,US00912XAT19,NaN,AIR LEASE CORP,AL 3 09/15/23,GLOBAL,Corp,Corp,AL 3 09/15/23
3,026874156,US0268741560,NaN,NotFound,NotFound,NotFound,NotFound,NotFound,NotFound
4,052476F52,NaN,NaN,AUSTIN WTR-REF,TX AUSUTL 5 05/15/2026,FIXED,Muni,Muni,NaN
...,...,...,...,...,...,...,...,...,...
19995,01212PAJ5,NaN,NaN,ALBANY CNTY,NY ALB 5 06/01/2023,FIXED,Muni,Muni,NaN
19996,01973R101,US01973R1014,ALSN,ALLISON TRANSMISSION HOLDING,ALSN,Common Stock,Common Stock,Equity,ALSN
19997,025816109,US0258161092,AXP,AMERICAN EXPRESS CO,AXP,Common Stock,Common Stock,Equity,AXP
19998,025816BS7,US025816BS73,NaN,AMERICAN EXPRESS CO,AXP 3.4 02/27/23,GLOBAL,Corp,Corp,AXP 3.4 02/27/23


In [43]:
isinmerge(merged_schwab, schwab_isin)

,cusip,isin,tickerSymbol,name,ticker,type1,type2,sector,description
0,011642105,US0116421050,ALRM,ALARM.COM HOLDINGS INC,ALRM,Common Stock,Common Stock,Equity,ALRM
1,089302103,US0893021032,BIG,BIG LOTS INC,BIG,Common Stock,Common Stock,Equity,BIG
2,29355A107,US29355A1079,ENPH,ENPHASE ENERGY INC,ENPH,Common Stock,Common Stock,Equity,ENPH
3,35671D857,US35671D8570,FCX,FREEPORT-MCMORAN INC,FCX,Common Stock,Common Stock,Equity,FCX
4,428567101,US4285671016,HIBB,HIBBETT SPORTS INC,HIBB,Common Stock,Common Stock,Equity,HIBB
...,...,...,...,...,...,...,...,...,...
19995,40434L105,US40434L1052,HPQ,HP INC,7HP,Common Stock,Common Stock,Equity,7HP
19996,459200101,US4592001014,IBM,INTL BUSINESS MACHINES CORP,IBM,Common Stock,Common Stock,Equity,IBM
19997,46120E602,US46120E6023,ISRG,INTUITIVE SURGICAL INC,ISRG,Common Stock,Common Stock,Equity,ISRG
19998,49456B101,US49456B1017,KMI,KINDER MORGAN INC,KMI,Common Stock,Common Stock,Equity,KMI


In [44]:
merged_mssb.to_csv('/Users/aeriliu/Documents/Columbia/Spring2021/IEOR4511/New Vehicle Data/merged_mssb_cusip+isin.csv')
merged_schwab.to_csv('/Users/aeriliu/Documents/Columbia/Spring2021/IEOR4511/New Vehicle Data/merged_schwab_cusip+isin.csv')

In [45]:
tickermerge(merged_mssb, mssb_ticker)

,cusip,isin,tickerSymbol,name,ticker,type1,type2,sector,description
0,001084102,US0010841023,AGCO,AGCO CORP,AGCO,Common Stock,Common Stock,Equity,AGCO
1,00751Y106,US00751Y1064,AAP,ADVANCE AUTO PARTS INC,AWN,Common Stock,Common Stock,Equity,AWN
2,00912XAT1,US00912XAT19,NaN,AIR LEASE CORP,AL 3 09/15/23,GLOBAL,Corp,Corp,AL 3 09/15/23
3,026874156,US0268741560,NaN,NotFound,NotFound,NotFound,NotFound,NotFound,NotFound
4,052476F52,NaN,NaN,AUSTIN WTR-REF,TX AUSUTL 5 05/15/2026,FIXED,Muni,Muni,NaN
...,...,...,...,...,...,...,...,...,...
19995,01212PAJ5,NaN,NaN,ALBANY CNTY,NY ALB 5 06/01/2023,FIXED,Muni,Muni,NaN
19996,01973R101,US01973R1014,ALSN,ALLISON TRANSMISSION HOLDING,ALSN,Common Stock,Common Stock,Equity,ALSN
19997,025816109,US0258161092,AXP,AMERICAN EXPRESS CO,AXP,Common Stock,Common Stock,Equity,AXP
19998,025816BS7,US025816BS73,NaN,AMERICAN EXPRESS CO,AXP 3.4 02/27/23,GLOBAL,Corp,Corp,AXP 3.4 02/27/23


In [46]:
tickermerge(merged_schwab, schwab_ticker)

,cusip,isin,tickerSymbol,name,ticker,type1,type2,sector,description
0,011642105,US0116421050,ALRM,ALARM.COM HOLDINGS INC,ALRM,Common Stock,Common Stock,Equity,ALRM
1,089302103,US0893021032,BIG,BIG LOTS INC,BIG,Common Stock,Common Stock,Equity,BIG
2,29355A107,US29355A1079,ENPH,ENPHASE ENERGY INC,ENPH,Common Stock,Common Stock,Equity,ENPH
3,35671D857,US35671D8570,FCX,FREEPORT-MCMORAN INC,FCX,Common Stock,Common Stock,Equity,FCX
4,428567101,US4285671016,HIBB,HIBBETT SPORTS INC,HIBB,Common Stock,Common Stock,Equity,HIBB
...,...,...,...,...,...,...,...,...,...
19995,40434L105,US40434L1052,HPQ,HP INC,7HP,Common Stock,Common Stock,Equity,7HP
19996,459200101,US4592001014,IBM,INTL BUSINESS MACHINES CORP,IBM,Common Stock,Common Stock,Equity,IBM
19997,46120E602,US46120E6023,ISRG,INTUITIVE SURGICAL INC,ISRG,Common Stock,Common Stock,Equity,ISRG
19998,49456B101,US49456B1017,KMI,KINDER MORGAN INC,KMI,Common Stock,Common Stock,Equity,KMI


In [47]:
merged_mssb.to_csv('/Users/aeriliu/Documents/Columbia/Spring2021/IEOR4511/New Vehicle Data/figi0018.csv')
merged_schwab.to_csv('/Users/aeriliu/Documents/Columbia/Spring2021/IEOR4511/New Vehicle Data/figi0023.csv')

# Correcting notfound

In [72]:
merged_mssb=pd.read_csv('/Users/aeriliu/Documents/Columbia/Spring2021/IEOR4511/New Vehicle Data/figi0018.csv')

In [73]:
merged_schwab=pd.read_csv('/Users/aeriliu/Documents/Columbia/Spring2021/IEOR4511/New Vehicle Data/figi0023.csv')

In [74]:
merged_mssb=merged_mssb.drop(columns=['Unnamed: 0'])

In [75]:
merged_schwab=merged_schwab.drop(columns=['Unnamed: 0'])

In [77]:
mssb_notfound=merged_mssb.loc[merged_mssb['name'] == 'NotFound']

In [78]:
schwab_notfound=merged_schwab.loc[merged_schwab['name'] == 'NotFound']

In [79]:
mssb_notfound

,cusip,isin,tickerSymbol,name,ticker,type1,type2,sector,description
3,026874156,US0268741560,NaN,NotFound,NotFound,NotFound,NotFound,NotFound,NotFound
492,E6R44C9Q9,NaN,NaN,NotFound,NotFound,NotFound,NotFound,NotFound,NotFound
608,602366WV4,NaN,NaN,NotFound,NotFound,NotFound,NotFound,NotFound,NotFound
727,19687N109,NaN,NaN,NotFound,NotFound,NotFound,NotFound,NotFound,NotFound
828,696550ZX3,NaN,NaN,NotFound,NotFound,NotFound,NotFound,NotFound,NotFound
...,...,...,...,...,...,...,...,...,...
18713,G9T45R126,NaN,AAGS,NotFound,NotFound,NotFound,NotFound,NotFound,NotFound
19240,E6R44C9Q9,NaN,NaN,NotFound,NotFound,NotFound,NotFound,NotFound,NotFound
19902,584CVR997,NaN,NaN,NotFound,NotFound,NotFound,NotFound,NotFound,NotFound
19909,602366WV4,NaN,NaN,NotFound,NotFound,NotFound,NotFound,NotFound,NotFound


In [80]:
schwab_notfound

,cusip,isin,tickerSymbol,name,ticker,type1,type2,sector,description
479,021ESC017,NaN,NaN,NotFound,NotFound,NotFound,NotFound,NotFound,NotFound
510,NaN,NaN,GLD 210319P00167000,NotFound,NotFound,NotFound,NotFound,NotFound,NotFound
562,NaN,NaN,GLD 210319P00165000,NotFound,NotFound,NotFound,NotFound,NotFound,NotFound
606,808516306,NaN,SCGB1,NotFound,NotFound,NotFound,NotFound,NotFound,NotFound
608,NaN,NaN,GLD 210319C00205000,NotFound,NotFound,NotFound,NotFound,NotFound,NotFound
...,...,...,...,...,...,...,...,...,...
19869,743CVR037,NaN,NaN,NotFound,NotFound,NotFound,NotFound,NotFound,NotFound
19870,86664Q108,NaN,NaN,NotFound,NotFound,NotFound,NotFound,NotFound,NotFound
19921,NaN,NaN,NaN,NotFound,NotFound,NotFound,NotFound,NotFound,NotFound
19922,NaN,NaN,NaN,NotFound,NotFound,NotFound,NotFound,NotFound,NotFound


In [56]:
def figi_from_notfound(df):

    cusip_list = df['cusip'].unique()

    cusipinfo = getOutput('ID_CUSIP', cusip_list.tolist())

    cusip_df = construct_df('cusip', cusipinfo)

    current_time = datetime.now().strftime("%H%M%S")
    cusip_df.to_csv(DIR + "/cusip_type_figi_notfound" + current_time + ".csv")

    return [cusip_df]

In [57]:
figi_from_notfound(mssb_notfound)

[       cusip                   name                  ticker     type1  \
 0  026874156               NotFound                NotFound  NotFound   
 1  E6R44C9Q9               NotFound                NotFound  NotFound   
 2  602366WV4        MILWAUKEE-N4&B5     WI MLW 5 04/01/2025     FIXED   
 3  19687N109               NotFound                NotFound  NotFound   
 4  696550ZX3  PALM BEACH CO SCH BRD  FL PALEDU 5 08/01/2027     FIXED   
 5  061871901               NotFound                NotFound  NotFound   
 6  679111YV0      OK TURNPIKE-D-REF  OK OKSTRN 5 01/01/2024     FIXED   
 7  G9T45R126               NotFound                NotFound  NotFound   
 8  584CVR997               NotFound                NotFound  NotFound   
 
       type2    sector description  
 0  NotFound  NotFound    NotFound  
 1  NotFound  NotFound    NotFound  
 2      Muni      Muni        None  
 3  NotFound  NotFound    NotFound  
 4      Muni      Muni        None  
 5  NotFound  NotFound    NotFound  

In [58]:
figi_from_notfound(schwab_notfound)

10
20
30
40
50
60
70


[        cusip                   name                  ticker     type1  \
 0   021ESC017               NotFound                NotFound  NotFound   
 1         NaN               NotFound                NotFound  NotFound   
 2   808516306               NotFound                NotFound  NotFound   
 3   13067WLU4  CALIFORNIA ST DEPT OF  CA CASWTR 5 12/01/2026     FIXED   
 4   544647BP4  LOS ANGELES CA UNIF S  CA LOSSCD 5 07/01/2032     FIXED   
 ..        ...                    ...                     ...       ...   
 67  317992600               NotFound                NotFound  NotFound   
 68  002422301               NotFound                NotFound  NotFound   
 69  743CVR037               NotFound                NotFound  NotFound   
 70  86664Q108               NotFound                NotFound  NotFound   
 71  921ESC034               NotFound                NotFound  NotFound   
 
        type2    sector description  
 0   NotFound  NotFound    NotFound  
 1   NotFound  NotFoun

In [60]:
notfound_18 = pd.read_csv('/Users/aeriliu/Documents/Columbia/Spring2021/IEOR4511/cusip_type_figi_notfound_0018.csv')
notfound_23 = pd.read_csv('/Users/aeriliu/Documents/Columbia/Spring2021/IEOR4511/cusip_type_figi_notfound_0023.csv')

In [61]:
notfound_18=notfound_18.drop(columns=['Unnamed: 0'])
notfound_23=notfound_23.drop(columns=['Unnamed: 0'])

In [62]:
notfound_23

,cusip,name,ticker,type1,type2,sector,description
0,021ESC017,NotFound,NotFound,NotFound,NotFound,NotFound,NotFound
1,NaN,NotFound,NotFound,NotFound,NotFound,NotFound,NotFound
2,808516306,NotFound,NotFound,NotFound,NotFound,NotFound,NotFound
3,13067WLU4,CALIFORNIA ST DEPT OF,CA CASWTR 5 12/01/2026,FIXED,Muni,Muni,NaN
4,544647BP4,LOS ANGELES CA UNIF S,CA LOSSCD 5 07/01/2032,FIXED,Muni,Muni,NaN
...,...,...,...,...,...,...,...
67,317992600,NotFound,NotFound,NotFound,NotFound,NotFound,NotFound
68,002422301,NotFound,NotFound,NotFound,NotFound,NotFound,NotFound
69,743CVR037,NotFound,NotFound,NotFound,NotFound,NotFound,NotFound
70,86664Q108,NotFound,NotFound,NotFound,NotFound,NotFound,NotFound


In [63]:
notfound_18

,cusip,name,ticker,type1,type2,sector,description
0,026874156,NotFound,NotFound,NotFound,NotFound,NotFound,NotFound
1,E6R44C9Q9,NotFound,NotFound,NotFound,NotFound,NotFound,NotFound
2,602366WV4,MILWAUKEE-N4&B5,WI MLW 5 04/01/2025,FIXED,Muni,Muni,NaN
3,19687N109,NotFound,NotFound,NotFound,NotFound,NotFound,NotFound
4,696550ZX3,PALM BEACH CO SCH BRD,FL PALEDU 5 08/01/2027,FIXED,Muni,Muni,NaN
5,061871901,NotFound,NotFound,NotFound,NotFound,NotFound,NotFound
6,679111YV0,OK TURNPIKE-D-REF,OK OKSTRN 5 01/01/2024,FIXED,Muni,Muni,NaN
7,G9T45R126,NotFound,NotFound,NotFound,NotFound,NotFound,NotFound
8,584CVR997,NotFound,NotFound,NotFound,NotFound,NotFound,NotFound


In [64]:
notfound_18.set_index('cusip', inplace=True)
notfound_23.set_index('cusip', inplace=True)

In [76]:
merged_mssb

,cusip,isin,tickerSymbol,name,ticker,type1,type2,sector,description
0,001084102,US0010841023,AGCO,AGCO CORP,AGCO,Common Stock,Common Stock,Equity,AGCO
1,00751Y106,US00751Y1064,AAP,ADVANCE AUTO PARTS INC,AWN,Common Stock,Common Stock,Equity,AWN
2,00912XAT1,US00912XAT19,NaN,AIR LEASE CORP,AL 3 09/15/23,GLOBAL,Corp,Corp,AL 3 09/15/23
3,026874156,US0268741560,NaN,NotFound,NotFound,NotFound,NotFound,NotFound,NotFound
4,052476F52,NaN,NaN,AUSTIN WTR-REF,TX AUSUTL 5 05/15/2026,FIXED,Muni,Muni,NaN
...,...,...,...,...,...,...,...,...,...
19995,01212PAJ5,NaN,NaN,ALBANY CNTY,NY ALB 5 06/01/2023,FIXED,Muni,Muni,NaN
19996,01973R101,US01973R1014,ALSN,ALLISON TRANSMISSION HOLDING,ALSN,Common Stock,Common Stock,Equity,ALSN
19997,025816109,US0258161092,AXP,AMERICAN EXPRESS CO,AXP,Common Stock,Common Stock,Equity,AXP
19998,025816BS7,US025816BS73,NaN,AMERICAN EXPRESS CO,AXP 3.4 02/27/23,GLOBAL,Corp,Corp,AXP 3.4 02/27/23


In [81]:
for i in range(len(merged_mssb)):
    if merged_mssb['name'][i]=='NotFound':
        merged_mssb['name'][i]=notfound_18.loc[merged_mssb['cusip'][i], 'name']
        merged_mssb['ticker'][i]=notfound_18.loc[merged_mssb['cusip'][i], 'ticker']
        merged_mssb['type1'][i]=notfound_18.loc[merged_mssb['cusip'][i], 'type1']
        merged_mssb['type2'][i]=notfound_18.loc[merged_mssb['cusip'][i], 'type2']
        merged_mssb['sector'][i]=notfound_18.loc[merged_mssb['cusip'][i], 'sector']
        merged_mssb['description'][i]=notfound_18.loc[merged_mssb['cusip'][i], 'description']

In [82]:
for i in range(len(merged_schwab)):
    if merged_schwab['name'][i]=='NotFound':
        merged_schwab['name'][i]=notfound_23.loc[merged_schwab['cusip'][i], 'name']
        merged_schwab['ticker'][i]=notfound_23.loc[merged_schwab['cusip'][i], 'ticker']
        merged_schwab['type1'][i]=notfound_23.loc[merged_schwab['cusip'][i], 'type1']
        merged_schwab['type2'][i]=notfound_23.loc[merged_schwab['cusip'][i], 'type2']
        merged_schwab['sector'][i]=notfound_23.loc[merged_schwab['cusip'][i], 'sector']
        merged_schwab['description'][i]=notfound_23.loc[merged_schwab['cusip'][i], 'description']

In [83]:
figi_0018_final = merged_mssb
figi_0023_final = merged_schwab

In [84]:
figi_0018_final

,cusip,isin,tickerSymbol,name,ticker,type1,type2,sector,description
0,001084102,US0010841023,AGCO,AGCO CORP,AGCO,Common Stock,Common Stock,Equity,AGCO
1,00751Y106,US00751Y1064,AAP,ADVANCE AUTO PARTS INC,AWN,Common Stock,Common Stock,Equity,AWN
2,00912XAT1,US00912XAT19,NaN,AIR LEASE CORP,AL 3 09/15/23,GLOBAL,Corp,Corp,AL 3 09/15/23
3,026874156,US0268741560,NaN,NotFound,NotFound,NotFound,NotFound,NotFound,NotFound
4,052476F52,NaN,NaN,AUSTIN WTR-REF,TX AUSUTL 5 05/15/2026,FIXED,Muni,Muni,NaN
...,...,...,...,...,...,...,...,...,...
19995,01212PAJ5,NaN,NaN,ALBANY CNTY,NY ALB 5 06/01/2023,FIXED,Muni,Muni,NaN
19996,01973R101,US01973R1014,ALSN,ALLISON TRANSMISSION HOLDING,ALSN,Common Stock,Common Stock,Equity,ALSN
19997,025816109,US0258161092,AXP,AMERICAN EXPRESS CO,AXP,Common Stock,Common Stock,Equity,AXP
19998,025816BS7,US025816BS73,NaN,AMERICAN EXPRESS CO,AXP 3.4 02/27/23,GLOBAL,Corp,Corp,AXP 3.4 02/27/23


In [85]:
figi_0023_final

,cusip,isin,tickerSymbol,name,ticker,type1,type2,sector,description
0,011642105,US0116421050,ALRM,ALARM.COM HOLDINGS INC,ALRM,Common Stock,Common Stock,Equity,ALRM
1,089302103,US0893021032,BIG,BIG LOTS INC,BIG,Common Stock,Common Stock,Equity,BIG
2,29355A107,US29355A1079,ENPH,ENPHASE ENERGY INC,ENPH,Common Stock,Common Stock,Equity,ENPH
3,35671D857,US35671D8570,FCX,FREEPORT-MCMORAN INC,FCX,Common Stock,Common Stock,Equity,FCX
4,428567101,US4285671016,HIBB,HIBBETT SPORTS INC,HIBB,Common Stock,Common Stock,Equity,HIBB
...,...,...,...,...,...,...,...,...,...
19995,40434L105,US40434L1052,HPQ,HP INC,7HP,Common Stock,Common Stock,Equity,7HP
19996,459200101,US4592001014,IBM,INTL BUSINESS MACHINES CORP,IBM,Common Stock,Common Stock,Equity,IBM
19997,46120E602,US46120E6023,ISRG,INTUITIVE SURGICAL INC,ISRG,Common Stock,Common Stock,Equity,ISRG
19998,49456B101,US49456B1017,KMI,KINDER MORGAN INC,KMI,Common Stock,Common Stock,Equity,KMI


In [86]:
figi_0018_final.loc[figi_0018_final['name'] == 'NotFound']

,cusip,isin,tickerSymbol,name,ticker,type1,type2,sector,description
3,026874156,US0268741560,NaN,NotFound,NotFound,NotFound,NotFound,NotFound,NotFound
492,E6R44C9Q9,NaN,NaN,NotFound,NotFound,NotFound,NotFound,NotFound,NotFound
727,19687N109,NaN,NaN,NotFound,NotFound,NotFound,NotFound,NotFound,NotFound
888,061871901,NaN,BDPS,NotFound,NotFound,NotFound,NotFound,NotFound,NotFound
1566,G9T45R126,NaN,AAGS,NotFound,NotFound,NotFound,NotFound,NotFound,NotFound
1829,584CVR997,NaN,NaN,NotFound,NotFound,NotFound,NotFound,NotFound,NotFound
3009,19687N109,NaN,NaN,NotFound,NotFound,NotFound,NotFound,NotFound,NotFound
3504,E6R44C9Q9,NaN,NaN,NotFound,NotFound,NotFound,NotFound,NotFound,NotFound
3699,026874156,US0268741560,NaN,NotFound,NotFound,NotFound,NotFound,NotFound,NotFound
3842,G9T45R126,NaN,AAGS,NotFound,NotFound,NotFound,NotFound,NotFound,NotFound


In [87]:
figi_0023_final.loc[figi_0023_final['name'] == 'NotFound']

,cusip,isin,tickerSymbol,name,ticker,type1,type2,sector,description
479,021ESC017,NaN,NaN,NotFound,NotFound,NotFound,NotFound,NotFound,NotFound
510,NaN,NaN,GLD 210319P00167000,NotFound,NotFound,NotFound,NotFound,NotFound,NotFound
562,NaN,NaN,GLD 210319P00165000,NotFound,NotFound,NotFound,NotFound,NotFound,NotFound
606,808516306,NaN,SCGB1,NotFound,NotFound,NotFound,NotFound,NotFound,NotFound
608,NaN,NaN,GLD 210319C00205000,NotFound,NotFound,NotFound,NotFound,NotFound,NotFound
...,...,...,...,...,...,...,...,...,...
19869,743CVR037,NaN,NaN,NotFound,NotFound,NotFound,NotFound,NotFound,NotFound
19870,86664Q108,NaN,NaN,NotFound,NotFound,NotFound,NotFound,NotFound,NotFound
19921,NaN,NaN,NaN,NotFound,NotFound,NotFound,NotFound,NotFound,NotFound
19922,NaN,NaN,NaN,NotFound,NotFound,NotFound,NotFound,NotFound,NotFound


In [88]:
figi_0018_final.to_csv('/Users/aeriliu/Documents/Columbia/Spring2021/IEOR4511/figi_0018_final2.csv')
figi_0023_final.to_csv('/Users/aeriliu/Documents/Columbia/Spring2021/IEOR4511/figi_0023_final2.csv')

# 0023 searching

In [212]:
notfound0023=figi_0023_final.loc[figi_0023_final['name'] == 'NotFound']

In [221]:
notfound0023['cusip'].unique()

array(['84990175', '317992600', nan, '34459U504', '370ESCBB0',
       '939ESC968', '535ESC107', '716013107', 'G3663X102', 'G5463T116',
       'G0112R116', '9.353244E+06', '12509J109', '9.7445E+66',
       '884510NT1', '67057N951', '524ESC100', '63007Y106', '131910101',
       '137801106', '135305100', '88673X207', '645ESC9A4', '808516207',
       '808516603', '656568508', '808516306', '885218800', '60254U100',
       '943526608', '4.42331E+80', '091482GB9', '370ESC121', '165ESCAB0',
       '021ESC017', '83587V108', '370ESCBS3', '370ESCBW4', '868157306',
       '165ESCAQ7', '146280300', '379570203', '370ESCAJ4', '284CVR016',
       '629998659', '46267T206', '641CVR014', '00772A616', '00772C810',
       '00772C836', '00772R114', '00773A698', '00773E310', '7.72E+115',
       '00773A656', '76123W101', '9.7464E+102', '4.0588E+31',
       '4.0588E+47', '1.89342E+51', '1.322856E+08', '4.520017E+10',
       '5.288283E+10', '5.426903E+06', '494656T35', '5.44495E+96',
       '6.461396E+06', '6.8

In [216]:
def getOutput2(IdType, idList):
    """
    IdType: str
        String that indicates the id type. Use defined idTypes
    idList: list
        list that contains ids of a specific type
    Return
        dict: key=>id , value=> list of two types from openfigi
    """
    idInfo = {}
    jobsList = []
    counter = 0
    notwork = []
    for id in idList:
        counter += 1
        jobsList.append(id)
        if counter % 10 == 0:
            print(counter)
            time.sleep(2)
        try:
            requestJobs = construct_jobs_mapping({IdType:jobsList})
            output = search_jobs(requestJobs)
            idInfo[jobsList[0]] = parse_output_extract_first(output)
        except:
            idInfo[jobsList[0]] = ["NotFound", "NotFound", "NotFound", "NotFound", "NotFound", "NotFound"]
            # notwork.append(id)
        # for i in len(output):
        # cusipinfo[jobsList[i]] = parse_output_extract_first(output)
        jobsList = []
    return idInfo

"""
Get figi type from pandas dataframe
"""
def figi_from_pd2(df):
    """
    df: pandas dataframe
        A pandas dataframe with three columns of different ids
    IdType: list
        list of IdType. Prefer 'Cusip', 'Isin', 'TickerSymbol'.
    Return
        
    """
    cusip_list = df['cusip'].unique()

    cusipinfo = getOutput2('ID_CUSIP', cusip_list.tolist())
    
    cusip_df = construct_df('cusip', cusipinfo)
    
    current_time = datetime.now().strftime("%H%M%S")
    cusip_df.to_csv(DIR + "/cusip_type_figi" + current_time + ".csv")

    return cusip_df

In [217]:
figi_from_pd2(notfound0023)

10
20
30
40
50
60
70
80
90
100
110
120
130
140
150
160
170
180
190
200


,cusip,name,ticker,type1,type2,sector,description
0,84990175,NotFound,NotFound,NotFound,NotFound,NotFound,NotFound
1,317992600,NotFound,NotFound,NotFound,NotFound,NotFound,NotFound
2,NaN,NotFound,NotFound,NotFound,NotFound,NotFound,NotFound
3,34459U504,NotFound,NotFound,NotFound,NotFound,NotFound,NotFound
4,370ESCBB0,NotFound,NotFound,NotFound,NotFound,NotFound,NotFound
...,...,...,...,...,...,...,...
202,525ESCLC1,NotFound,NotFound,NotFound,NotFound,NotFound,NotFound
203,P37307108,NotFound,NotFound,NotFound,NotFound,NotFound,NotFound
204,50183L107,NotFound,NotFound,NotFound,NotFound,NotFound,NotFound
205,502003106,NotFound,NotFound,NotFound,NotFound,NotFound,NotFound


In [224]:
notfound23=pd.read_csv('/Users/aeriliu/Documents/Columbia/Spring2021/IEOR4511/New Vehicle Data/Figi_Scrap/notfound0023.csv')

In [225]:
notfound23=notfound23.drop(columns=['Unnamed: 0'])

In [227]:
notfound23.set_index('cusip', inplace=True)

In [228]:
for i in range(len(figi_0023_final)):
    if figi_0023_final['name'][i]=='NotFound':
        figi_0023_final['name'][i]=notfound23.loc[figi_0023_final['cusip'][i], 'name']
        figi_0023_final['ticker'][i]=notfound23.loc[figi_0023_final['cusip'][i], 'ticker']
        figi_0023_final['type1'][i]=notfound23.loc[figi_0023_final['cusip'][i], 'type1']
        figi_0023_final['type2'][i]=notfound23.loc[figi_0023_final['cusip'][i], 'type2']
        figi_0023_final['sector'][i]=notfound23.loc[figi_0023_final['cusip'][i], 'sector']
        figi_0023_final['description'][i]=notfound23.loc[figi_0023_final['cusip'][i], 'description']

In [229]:
figi_0023_final.to_csv('/Users/aeriliu/Documents/Columbia/Spring2021/IEOR4511/New Vehicle Data/Figi_Scrap/figi_0023_final.csv')

In [230]:
figi_0023_final.loc[figi_0023_final['name'] == 'NotFound']

,cusip,isin,tickerSymbol,name,ticker,type1,type2,sector,description
190,84990175,NaN,NaN,NotFound,NotFound,NotFound,NotFound,NotFound,NotFound
191,84990175,NaN,NaN,NotFound,NotFound,NotFound,NotFound,NotFound,NotFound
217,317992600,NaN,NaN,NotFound,NotFound,NotFound,NotFound,NotFound,NotFound
258,84990175,NaN,NaN,NotFound,NotFound,NotFound,NotFound,NotFound,NotFound
320,84990175,NaN,NaN,NotFound,NotFound,NotFound,NotFound,NotFound,NotFound
...,...,...,...,...,...,...,...,...,...
270326,6.500357E+08,NaN,NaN,NotFound,NotFound,NotFound,NotFound,NotFound,NotFound
270474,4.212905E+09,NaN,NaN,NotFound,NotFound,NotFound,NotFound,NotFound,NotFound
270487,4.78497E+35,NaN,NaN,NotFound,NotFound,NotFound,NotFound,NotFound,NotFound
271174,93317Q105,US93317Q1058,NaN,NotFound,NotFound,NotFound,NotFound,NotFound,NotFound
